* Conv
* MaxPool
* Dropout
* Data Augmentation 
* Activation - ReLu

### Architecture
* 1st layer Conv2D (kernel_size = (11,11) , 96 filters and strides = 4
* Next layer MaxPoolD (pool_size = (3,3) and strides = 2
* 2nd conolutional layer  Conv2D (kernel_size = (5,5) , 256 filters and padding = 2
* Followed by MaxPool2D (pool_size = (3,3) and strides = 2
* 3rd conolutional layer  Conv2D (kernel_size = (3,3) , 384 filters and padding = 1
* Followed by MaxPool2D (pool_size = (3,3) and strides = 2
* FC 4096
* FC 4096
* Output layer  - softmax 1000

if padding==same ==0

**(n+2p-f/s)+1**


**1st conv:**

(227+2(0)-11/4)+1 = 55 

55x55x96 = 290400

**1st max pool:**

((55+0-3)/2) +1 = 27




In [1]:
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization,MaxPool2D
from keras.preprocessing.image import ImageDataGenerator 
from keras.datasets import cifar10
import numpy as np

In [2]:
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd fully connected layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(10))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

In [3]:
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 8, 96)          34944     
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 96)         384       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 8, 8, 96)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 4, 96)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 256)         614656    
                                                                 
 batch_normalization_1 (Batc  (None, 4, 4, 256)        1

In [4]:
(X_train,Y_train),(X_test,Y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 8s 0us/step


In [5]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size = 0.3)

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(35000, 32, 32, 3)
(35000, 1)
(15000, 32, 32, 3)
(15000, 1)


In [7]:
from sklearn.utils.multiclass import unique_labels
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)
y_test = np_utils.to_categorical(Y_test)

In [8]:
train_datagen = ImageDataGenerator(
    # rescale=1./255,
    rotation_range = 2,
    zoom_range=1,
    horizontal_flip=True,    
)

val_datagen = ImageDataGenerator(
    # rescale=1./255,
    rotation_range = 2,
    zoom_range=1,
    horizontal_flip=True,    
)
test_datagen = ImageDataGenerator(
    # rescale=1./255,
    rotation_range = 2,
    zoom_range=1,
    horizontal_flip=True,    
)
train_datagen.fit(x_train)
test_datagen.fit(X_test)
val_datagen.fit(x_val)

In [9]:
AlexNet.compile(optimizer='RMSprop',loss = 'categorical_crossentropy',metrics=['accuracy'])

In [18]:
AlexNet.fit_generator(train_datagen.flow(x_train,y_train,batch_size=32),epochs=3,steps_per_epoch=X_train.shape[0]//32,validation_data=val_datagen.flow(x_val,y_val,batch_size=32),validation_steps=250,verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/3
1562/1562 [==============================] - 74s 37ms/step - loss: 2.0138 - accuracy: 0.2367 - val_loss: 2.0087 - val_accuracy: 0.2406
